In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
events=pd.read_excel(r"C:\Users\racha\Documents\EXL_case\Data & Submission Template\Events Data.xlsx")
teams=pd.read_excel(r"C:\Users\racha\Documents\EXL_case\Data & Submission Template\Game Metadata.xlsx")
game_meta_data=pd.ExcelFile(r"C:\Users\racha\Documents\EXL_case\Data & Submission Template\Game Metadata.xlsx")
managers=pd.read_excel(game_meta_data,'managers')
manager_history=pd.read_excel(game_meta_data,'manager_history')
referees=pd.read_excel(game_meta_data,'referees')
features={'teams':['team_id','team_country','team_name','team_city'],
'events':['season','game_week','game_id','home_team_id','away_team_id','home_goals_scored','away_goals_scored','home_formation','away_formation','home_num_injuries','away_num_injuries','referee_id'],
'manager':['manager_id','manager_name','manager_badge','manager_age'],
'referees':['referee_id','referee_name','referee_country','referee_experience']}
toDrop=['home_team_id','away_team_id','home_team_manager_id','away_team_manager_id','referee_id']


In [3]:
events_teams_home=pd.merge(events[features['events']],teams[features['teams']],how='inner',left_on='home_team_id',right_on='team_id',suffixes=('', '_drop')).filter(regex='^(?!.*_drop)')
events_teams_home=events_teams_home.drop(['team_id'],axis=1)
events_teams_home.rename(columns = {'team_name':'home_team_name','team_city':'home_team_city','team_country':'home_team_country'}, inplace = True)
events_teams_home


,season,game_week,game_id,home_team_id,away_team_id,home_goals_scored,away_goals_scored,home_formation,away_formation,home_num_injuries,away_num_injuries,referee_id,home_team_country,home_team_name,home_team_city
0,2012,1,efl_00001,xmq321,1xyza3,1,0,four three three,four two four,0,2,brit12,Greece,Peloponnisos,Athens
1,2012,3,efl_00022,xmq321,ju$xzs,3,2,three four three,five four one,1,0,jurg43,Greece,Peloponnisos,Athens
2,2012,4,efl_00037,xmq321,6re#$3,0,2,three four three,four two four,0,0,aweds4,Greece,Peloponnisos,Athens
3,2012,5,efl_00049,xmq321,9hgy&,0,2,five three two,three four three,2,0,aweds4,Greece,Peloponnisos,Athens
4,2012,6,efl_00052,xmq321,l0p4ra,2,0,four three three,four two four,2,2,vsed21,Greece,Peloponnisos,Athens
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,2021,67,efl_07504,ju$xzs,rew#ff,1,3,five four one,three four three,0,2,vsed21,Italy,Marche,Verona
7596,2021,70,efl_07537,ju$xzs,zd78$,1,2,five four one,four two four,2,0,xhh46,Italy,Marche,Verona
7597,2021,71,efl_07549,ju$xzs,xuxa#,1,0,five three two,three four three,0,1,xysd4,Italy,Marche,Verona
7598,2021,73,efl_07566,ju$xzs,1xyza3,0,2,four three three,four three three,0,1,xlo9,Italy,Marche,Verona


In [4]:
events_teams_away=pd.merge(events_teams_home,teams[features['teams']],how='inner',left_on='away_team_id',right_on='team_id',suffixes=('', '_drop')).filter(regex='^(?!.*_drop)')
events_teams_away=events_teams_away.drop(['team_id'],axis=1)
events_teams_away.rename(columns = {'team_name':'away_team_name','team_city':'away_team_city','team_location':'away_team_location','team_country':'away_team_country','team_stadium':'away_team_stadium','stadium_capacity':'away_stadium_capacity'}, inplace = True)
events_referee=pd.merge(events_teams_away,referees[features['referees']],how='inner')
train_df=events_referee.copy()

In [5]:
train_df=pd.merge(train_df,manager_history,how='inner',left_on=['season','home_team_id'],right_on=['season','team_id'])
del train_df['team_id']
train_df.rename(columns = {'manager_id':'home_team_manager_id'}, inplace = True)
train_df=pd.merge(train_df,manager_history,how='inner',left_on=['season','away_team_id'],right_on=['season','team_id'])
del train_df['team_id']
train_df.rename(columns = {'manager_id':'away_team_manager_id'}, inplace = True)

In [6]:
train_df=pd.merge(train_df,managers[features['manager']],how='inner',left_on='home_team_manager_id',right_on='manager_id')
del train_df['manager_id']
train_df.rename(columns = {'manager_name':'home_team_manager_name','manager_badge':'home_team_manager_badge','manager_age':'home_team_manager_age'}, inplace = True)
train_df=pd.merge(train_df,managers[features['manager']],how='inner',left_on='away_team_manager_id',right_on='manager_id')
del train_df['manager_id']
train_df.rename(columns = {'manager_name':'away_team_manager_name','manager_badge':'away_team_manager_badge','manager_age':'away_team_manager_age'}, inplace = True)


In [7]:
train_df.columns

Index(['season', 'game_week', 'game_id', 'home_team_id', 'away_team_id',
       'home_goals_scored', 'away_goals_scored', 'home_formation',
       'away_formation', 'home_num_injuries', 'away_num_injuries',
       'referee_id', 'home_team_country', 'home_team_name', 'home_team_city',
       'away_team_country', 'away_team_name', 'away_team_city', 'referee_name',
       'referee_country', 'referee_experience', 'home_team_manager_id',
       'away_team_manager_id', 'home_team_manager_name',
       'home_team_manager_badge', 'home_team_manager_age',
       'away_team_manager_name', 'away_team_manager_badge',
       'away_team_manager_age'],
      dtype='object')

In [8]:
corr=train_df.corr()

C:\Users\racha\AppData\Local\Temp\ipykernel_42124\7171292.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr=train_df.corr()


No correlation among the 12 numeric variables

            Feature Selection

features: refree height, refree marital status, refree badge,manager country, team loc, team stadium, game date, all ID's

In [9]:
train_df=train_df.drop(toDrop,axis=1)

In [10]:
train_df

,season,game_week,game_id,home_goals_scored,away_goals_scored,home_formation,away_formation,home_num_injuries,away_num_injuries,home_team_country,...,away_team_city,referee_name,referee_country,referee_experience,home_team_manager_name,home_team_manager_badge,home_team_manager_age,away_team_manager_name,away_team_manager_badge,away_team_manager_age
0,2012,1,efl_00001,1,0,four three three,four two four,0,2,Greece,...,Glasgow,Andreas Kapino,Greece,18,Dogan Bakkal,TUR,59,Konstantinos Pelkas,GRE,45
1,2012,54,efl_00539,4,3,four two four,three four three,0,0,Greece,...,Glasgow,Aaron McGinn,Scotland,21,Dogan Bakkal,TUR,59,Konstantinos Pelkas,GRE,45
2,2020,16,efl_06234,2,3,four three three,five three two,2,0,Russia,...,Stockholm,Hector Kroos,Germany,16,Dogan Bakkal,TUR,59,Konstantinos Pelkas,GRE,45
3,2020,64,efl_06720,0,1,five three two,four five one,1,1,Russia,...,Stockholm,Kieran Adams,Scotland,20,Dogan Bakkal,TUR,59,Konstantinos Pelkas,GRE,45
4,2021,5,efl_06883,2,1,four four two,five four one,0,3,Russia,...,Stockholm,Aaron McGinn,Scotland,21,Dogan Bakkal,TUR,59,Konstantinos Pelkas,GRE,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,2016,41,efl_03442,3,2,four three three,four four two,0,2,Spain,...,Wakefield,Emre Neuer,Germany,17,Jack Tierney,SCO,53,Daniel Klopp,GER,53
7596,2016,25,efl_03282,1,0,three four three,three four three,3,1,Italy,...,Wakefield,Oleksander Zubkov,Ukraine,18,George Rota,GRE,50,Daniel Klopp,GER,53
7597,2016,65,efl_03689,4,3,three four three,four four two,2,2,Italy,...,Wakefield,Luis Matador,Spain,21,George Rota,GRE,50,Daniel Klopp,GER,53
7598,2016,52,efl_03558,2,1,four four two,four three three,0,1,Russia,...,Wakefield,Oleksander Zubkov,Ukraine,18,Mesut Gorkey,TUR,61,Daniel Klopp,GER,53
